In [17]:
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate


from sklearn.preprocessing import StandardScaler
import random
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
cancelation_df = pd.read_csv('../../raw_data/latlong_ohe.csv')

In [3]:
cancelation_df.shape

(714851, 25)

In [4]:
cancelation_df = cancelation_df.sample(50000)

In [5]:
cancelation_df.head()

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,ORIGIN_LONGITUDE,ORIGIN_LATITUDE,DESTINATION_LONGITUDE,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
545626,5,25,1,171.0,1069,956,0,-80.15275,26.07258,-73.77893,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
623697,11,3,2,110.0,612,1040,0,-82.53325,27.97547,-86.67818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
337885,7,16,4,125.0,795,1020,0,-102.20191,31.94253,-115.15233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
673100,2,2,1,116.0,552,1016,1,-87.90446,41.97960,-92.22425,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
692904,2,23,1,83.0,396,1028,1,-97.03720,32.89595,-99.46154,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#Borrar NaN
cancelation_df = cancelation_df.dropna()

In [7]:
X = cancelation_df.drop(columns = ['CANCELLED'])
y = cancelation_df['CANCELLED']

In [8]:
X.shape, y.shape

((49997, 24), (49997,))

In [9]:
y.value_counts()

CANCELLED
0    43800
1     6197
Name: count, dtype: int64

In [10]:
# STEP 0 - Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [11]:
#Standard Scaler
std_scaler = StandardScaler()
X_train_sc = std_scaler.fit_transform(X_train)
X_test_sc=std_scaler.transform(X_test)

In [24]:
# Instanciate the model with default parameter of n_estimators = 10
rfc_10 = RandomForestClassifier(random_state=0)

# fit the model
rfc_10.fit(X_train_sc, y_train)

# Train the model on the scaled Training data
cv_results = cross_validate(rfc_10, X_train_sc, y_train, cv=5, scoring=['precision'])

In [26]:
rfc_score = cv_results['test_precision'].mean()
rfc_score

0.8056133711026071

In [27]:
# Predict the Test set results
y_pred = rfc_10.predict(X_test_sc)

In [32]:
# Instanciate the model with default parameter of n_estimators = 100
rfc_100 = RandomForestClassifier(n_estimators=100, random_state=0)

# fit the model
rfc_100.fit(X_train_sc, y_train)

# Train the model on the scaled Training data
cv_results_100 = cross_validate(rfc_100, X_train_sc, y_train, cv=5, scoring=['precision'])


In [33]:
rfc_score_100 = cv_results_100['test_precision'].mean()
rfc_score_100

0.8056133711026071

In [34]:
# Predict the Test set results
y_pred_100 = rfc_100.predict(X_test_sc)

In [ ]:
from sklearn.model_selection import GridSearchCV
rfc_gs = RandomForestClassifier()

# n_estimators
n_estimators = [10, 25, 50, 100, 150]

# max_features
max_features = ['sqrt', 'log2', None]

# max_depth
max_depth = [3, 6, 9] 

# max_leaf_nodes
max_leaf_nodes = [3, 6, 9]

param_grid = dict(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, max_leaf_nodes=max_leaf_nodes) 

# defining parameter range
grid = GridSearchCV(rfc_gs, param_grid, cv=5, scoring='precision', verbose=1, n_jobs=-1)
  
# fitting the model for grid search
grid_search=grid.fit(X_train_sc, y_train)

In [39]:
print(grid_search.best_params_)

{'max_depth': 6, 'max_features': 'sqrt', 'max_leaf_nodes': 9, 'n_estimators': 10}


In [40]:
print(grid.best_score_)

0.8
